In [1]:
from core import *
from modelparamgrid import *

C:\Users\tikyn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\aeon\base\__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


<b>The whole model works in this order</b>
<ol>
    <li>clasp model - in: features ts [21][98][n] - out: prediction [21][98][m] - n>m</li>
    <ul>
        <li>21 video, ogni video 98 timeseries</li>
        <li>per ogni 98 timeseries, 98 prediction</li>
        <li>n = lunghezza timeseries</li>
        <li>m = numeri dei prediction</li>
    </ul>
    <li>split saliency - in: prediction [21][98][m] - out: pred_split [21][6][k][m]</li>
    <ul>
        <li>98 prediction divisi in 6 gruppi di k elementi. Possono esserci ripetizioni. 6*k > 98</li>
        <li>k = numero di prediction in un gruppo
    </ul>
    <li>combinesinglesaliency - in: pred_split[21][6][k][m] - out: salience_preds [21][6][x] - k>x</li>
    <ul>
        <li>ogni k prediction unite in 1 array e applicato MajorityVote</li>
        <li>x = MajorityVote genera la media di gruppi di prediction k</li>
    </ul>
    <li>union salience_preds - in: salience_preds [21][6][x] - out: union_pred [21][y] - y>x</li>
    <ul>
        <li>Le 6 salienze unite in un unico array</li>
        <li>y = unione di x
    </ul>
    <li>ignorezone - in: union_pred [21][y] - out: ignored_pred [21][a] - y>a</li>
    <ul>
        <li>Ignora le zone non annotate perchè fuori dal nostro use case</li>
        <li>a = numero di prediction dopo aver tolto i prediction dalle zone non annotate</li>
    </ul>
    <li>delnear - in: ignored_pred [21][a] - out: del_pred [21][b]</li>
    <ul>
        <li>Come MajorityVote ma lascia primo e ultimo del gruppo</li>
        <li>b = numero di prediction che delineano inizio e fine di movimenti</li>
    </ul>
    <li>DTW - in: del_pred [21][b] - out: dtw_pred [21][c] - b>c</li>
    <ul>
        <li>applico DTW per controllare se i prediction b hanno FP (false positive)</li>
        <li>c = numero di prediction con meno FP</li>
    </ul>
    <li>scoring - in: dtw_pred [21][c] - out: f1score [21]</li>
    <ul>
        <li>Per ogni prediction lungo c nei 21 video, calcolo lo score</li>
    </ul>
</ol>

This project is divided into three part, because each part takes a lot of days for computing the result:
<ol>
    <li>Clasp Model: (1) -> create all prediction from the clasp model</li>
    <li>Filtering: (2)(3)(4)(5)(6)(7) -> filters all prediction to remove false positives</li>
    <li>Scoring: (8) -> given the filtered prediction, gives an evaluation</li>
</ol>

<b>To find the perfect Hyperparameters we use gridsearch in this order</b>
<ol>
    <li>Clasp Model - in: [21][98][n] - out: [20][21][98][n]</li>
    <ul>
        <li>Gridsearch just for model generated 20 different clasp models</li>
    </ul>
    <li>Filtering - in: [20][21][98][n] - out: [20][243][21][c]</li>
    <ul>
        <li>Gridsearch for filtering generated 243 different results</li>
    </ul>
    <li>Scoring LOVO - in:[20][243][21][c] - out: f1score + hyperparameters</li>
    <ul>
        <li>Try everything and return an f1score and which hyperparameters are the best</li>
    </ul>
    

</ol>

<b> Clasp Model (1)</b>
We use ParameterGrid() to compute all the combination given the param_grid and we obtained <b>20</b> different models
<p>in: features ts [21][98][n] - out: prediction [20][21][98][m]</p>

In [ ]:
# all selected hyperparameters values to gridsearch
param_grid = {
    "window_size": ['suss'],
    "distance": ['euclidean_distance'],
    "score":['roc_auc','f1'],
    "early_stopping": [True,False],
    "k_neighbours":[1,3,5,7,9],
    "n_jobs":[4]
}


# len(ParameterGrid(param_grid)) = 20
for p_idx,params in enumerate(ParameterGrid(param_grid)):
    #excl_radius has to be larger than neighbours
    params["excl_radius"]=params["k_neighbours"]+2
    # cp = prediction of 1 timeseries 
    # cps = prediction for n timeseries
    # cpsl = prediction for n timeseries for m different hyperparameters 
    cpsl=[]
    for i in range(len(TIMESERIES)):
        df=ReadAndPreProcess(TIMESERIES[i])
        cp_all,cps,clasp=GetClasp2(df,list(range(len(FEATURES_NAME))),p_idx,i, **params)
        cpsl.append(cps)

    # COMMENTED TO AVOID OVERWRITE ALREADY COMPUTED FILE
    # save prediction of [20][21][98] = [param][video][feature]
    #with open("../cpsl_param_"+str(p_idx)+".pkl", "wb") as f:
        #pickle.dump(cpsl, f)


<b>Filtering (2)(3)(4)(5)(6)(7)</b>
The gridsearch isn't finished yet. Each step has its known set of parameters that needs to be found to have the optimal model. So in the filtering phase we gridsearch all values and generate all possible filtered prediction from each different. 

The computational costs is made by for loops:
<p>for each models = 19</p>
<p style="text-indent: 15px;">for each params = 243 </p>
<p style="text-indent: 30px;">for each videos = 21 </p>
<p style="text-indent: 45px;">for each features = 98</p>
<p>in:[20][21][98] - out:[20][21][c]</p>

In [ ]:
#load dataframe for 21 videos
with open("../dfl.pkl", "rb") as f:
    dfl = pickle.load(f)

#load groundtruth for 21 videos
with open("../gtl.pkl", "rb") as f:
    gtl = pickle.load(f)

In [ ]:
# From previous step, load 20 models in modelpred
modelpred=[]
for i in range(0,20):
    with open("../res/cpsl_param_"+str(i)+".pkl", "rb") as f:
        modelpred.append(pickle.load(f))

#20 model
#21 video
#98 feature

In [ ]:
#2. from 98 predictions, divide it into 6 groups
# Some prediction could be repeated in the groups
def SplitSaliency(pred):
    # 2. split features in 6 saliency groups
    # vel,acc,jerk,specific,generic
    # specific: other type of feature specific for that saliency
    # generic: other type of feature that describe the whole body movement
    ## generic features: 0,3
    ariel_el=[26,27,28,29,30,31,32,33,34,35,36,37,2,8,14,0,3]
    ariel = UnionCPS(*pred[ariel_el])
   
    strn_el=[38,39,40,41,42,43,44,45,46,47,48,49,1,9,0,3]
    strn = UnionCPS(*pred[strn_el])

    rhel_el=[50,51,52,53,54,55,56,57,58,59,60,61,7,11,16,17,18,21,0,3]
    rhel = UnionCPS(*pred[rhel_el])

    lhel_el=[62,63,64,65,66,67,68,69,70,71,72,73,6,10,16,17,19,20,0,3]
    lhel = UnionCPS(*pred[lhel_el])
    
    rplm_el=[74,75,76,77,78,79,80,81,82,83,84,85,5,13,15,23,25,0,3]
    rplm = UnionCPS(*pred[rplm_el])

    lplm_el=[86,87,88,89,90,91,92,93,94,95,96,97,4,12,15,22,24,0,3]
    lplm = UnionCPS(*pred[lplm_el])

    return [ariel,strn,rhel,lhel,rplm,lplm]

In [ ]:
# 3. given one list, determine the mean value in the group if it's within maj_margin
#    and decide if to compute the mean if the number of el in the group is over maj_thre
def CombineSingleSaliency(saliencies, maj_margin, maj_thre):
    res=[]
    for sal in saliencies:
        res.append(MajorityVoteCP(sal, maj_margin, maj_thre))
    return res
        


In [ ]:
# the filtering part for just one prediction.
# after we computed all prediction from 20 different models
# all prediction should be filtered to be combined and filtered to remove False Positive
# in: prediction
def PostFilterTogether(model_id,video_id,param_id,param_len,density,pred,gt, maj_margin, maj_thre, zone_margin, del_margin, dtw_threshold):
    """
        pred: 98 predictions of a single video
    """
    
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Start                                               ", end='\r')

    #2. from a list[98] divide it into 6 lists. Some prediction could be repeated into diff groups
    saliencies_raw = SplitSaliency(pred)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Split Saliency completed                            ", end='\r')
    #3. each group has different list of prediction. combine it into 1 and apply majorityvote
    saliencies = CombineSingleSaliency(saliencies_raw,maj_margin,maj_thre)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Combine Saliency completed                          ", end='\r')
    #4. from 6 list, combine into 1 list
    pred_united = UnionCPS(*saliencies)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Union completed                                     ", end='\r')
    #5. there is no annotated zone. remove prediction from that subset
    pred_ignored = IgnoreZone(video_id,pred_united,gt,zone_margin)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Ignore Zone completed                               ", end='\r')
    #6. apply motion fusion: in a group within del_margin, remove everything except 1 and last
    pred_del = delnear(pred_ignored,del_margin)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - delnear completed                                   ", end='\r')
    #7. check the results with DTW to remove potential FP
    pred_final=FPremoverDTW(density,pred_del,dtw_threshold)
    print(f"model:{model_id}/20 - param:{param_id}/{param_len} - video:{video_id}/21 - DTW completed                                       ", end='\r')
    # return a list of prediction
    return pred_final
    



In [ ]:
#Filtering part for all prediction from all 20 models
#gridsearch params for our filtering part
param_grid = {
    "maj_margin": [80,100,120],
    "maj_thre": [1,2,3],
    "zone_margin":[80,100,120],
    "del_margin": [80,100,120],
    "dtw_threshold":[80,100,120]
}



#for each model, for each params,
# compute for each video the filtering
# each leaf would be a prediction with a specific model and specific filter_param
model_res=[]
for model_id,model in enumerate(modelpred): #20 models predictions
    param_value=[]
    for p_idx,params in enumerate(ParameterGrid(param_grid)): #243
        video=[]
        for i in range(len(TIMESERIES)): #21 videos
            df = dfl[i]
            gt = gtl[i]
            res=PostFilterTogether(model_id,i,p_idx,len(ParameterGrid(param_grid)),df.iloc[:,3],np.array(model[i],dtype="object"),gt,params["maj_margin"],params["maj_thre"],params["zone_margin"],params["del_margin"],params["dtw_threshold"])
            video.append(res)
       
        param_value.append(video)
    model_res.append(param_value)

# COMMENTED TO AVOID OVERWRITE LOCAL FILES
# save the filtered prediction in local
#with open("../collection_models_prediction_filtered.pkl", "wb") as f:
#    pickle.dump(model_res, f)
    




<b>Scoring (8)</b>
here we take as input all the filtered results and performs a cross-validation with our predictions to see which hyperparameters for our model+filter is the best and then test it on the validation set

In [2]:
#load dataframe for 21 videos
with open("../dfl.pkl", "rb") as f:
    dfl = pickle.load(f)

#load groundtruth for 21 videos
with open("../gtl.pkl", "rb") as f:
    gtl = pickle.load(f)

In [3]:
# load the filtered prediction
with open("../collection_models_prediction_filtered.pkl", "rb") as f:
    collection_models_prediction_filtered = pickle.load(f)

In [4]:
models=collection_models_prediction_filtered
# model:20 params:243 video:21

In [ ]:
# Compute the f1 margin
# se il suss mi individua quella sezione dove ha proprietà statiche
# allora il prossimo changepoint non può essere minore di quello?
res=[]
for i in range(len(TIMESERIES)):
    df=dfl[i]
    for j in range(0,98):
        res.append(suss(df.iloc[:,j].values))
f1_margin=statistics.mean(res)



In [8]:
f1_margin=120

In [9]:
# LEAVE ONE VIDEO OUT - LOVO
scores=[]
for model_id,params in enumerate(models): #1 model = 243 params <- 20 model
    for p_idx,videos in enumerate(params): # 21 video = 1 param <- 243 params
        score_folds = []
        for video_idx in range(len(TIMESERIES)): # 1 prediction = 1 video <- 1 param = 21 video
            
            train_set = [i for i in range(len(TIMESERIES)) if i != video_idx]
            #f1score di 1 fold
            _,_,f1,_ = f1scoremargin(gtl,videos,train_set,f1_margin)
            score_folds.append(f1)

        scores.append({"model":model_id, "param":p_idx, "score":statistics.mean(score_folds)})

type_col=["model","param","score"]
resexcel=pd.DataFrame()
resexcel["type"]=type_col

best_models=[]
best_score=0
for i in scores:
    # if better, reset list and append
    if i["score"] > best_score:
        best_models=[]
        best_models.append(i)
        best_score = i["score"]
    # if equal, append to the list
    elif i["score"] == best_score:
        best_models.append(i)
        
best_scores=[]
for best_idx,best in enumerate(best_models):
    best_score=[]
    best_one=0

    for video_idx in range(len(TIMESERIES)):
        gt = gtl[video_idx]
        _,_,f1,_,_,_=f1scoremarginOne(gt,models[best["model"]][best["param"]][video_idx],f1_margin)
        best_score.append(f1)
        print(f'{TIMESERIES[video_idx]} {f1}')
    best_scores.append({"model":best["model"], "param":best["param"], "score":statistics.mean(best_score)})
    resexcel[best_idx]=[model_id,p_idx,statistics.mean(best_score)]

resexcel.to_excel("outputFile/LOVO.xlsx")
print("lovo")
for i in best_scores:
    print(i)






in\cora1_in.txt 0.6176470588235293
in\cora4_05_in.txt 0.7631578947368421
in\cora4_08_in.txt 0.9333333333333333
in\cora5_in.txt 0.5777777777777777
in\cora14_in.txt 0.6
in\marianne7_in.txt 0.75
in\marianne8_in.txt 0.782608695652174
in\marianne10_in.txt 0.8260869565217392
in\marianne18_in.txt 0.7857142857142856
in\marianne19_in.txt 0.6666666666666667
in\marianne24_in.txt 0.6363636363636364
in\marianne26_in.txt 0.7058823529411765
in\marianne41_in.txt 0.5714285714285715
in\marianne42_in.txt 0.6363636363636364
in\marianne43_in.txt 0.5977011494252873
in\marianne47_in.txt 0.7586206896551724
in\marianne48_in.txt 0.7096774193548386
in\muriel18_in.txt 0.7540983606557377
in\muriel26_in.txt 0.7786259541984734
in\muriel27_in.txt 0.7536231884057971
in\muriel30_in.txt 0.6363636363636364
lovo
{'model': 3, 'param': 0, 'score': 0.7067495840182053}


In [ ]:
# LEAVE ONE DANCER OUT - LODO
CORA=[0,1,2,3,4]
MARIANNE=[5,6,7,8,9,10,11,12,13,14,15,16]
MURIEL=[17,18,19,20]


scores=[]
for model_id,params in enumerate(models): #1 model = 243 params <- 20 model
    for p_idx,videos in enumerate(params): # 21 video = 1 param <- 243 params
        score_folds=[]
        
        #fold1
        TRAINING_1 = CORA+MARIANNE
        for ts in TRAINING_1:
            #get score each video for ts
            #compute score
            _,_,f1,_ = f1scoremargin(gtl,videos,TRAINING_1,f1_margin)
            score_folds.append(f1)
            """
            gt = gtl[ts]
            _,_,f1,_,_,_=f1scoremargin(gt,videos[ts],f1_margin)
            score_singlefold_eachvideo.append(f1)
        #media per training
        fold1_score = statistics.mean(score_singlefold_eachvideo)
        """
            
        #fold2
        TRAINING_2 = CORA+MURIEL
        for ts in TRAINING_2:
            _,_,f1,_ = f1scoremargin(gtl,videos,TRAINING_2,f1_margin)
            score_folds.append(f1)

        #fold3
        TRAINING_3 = MARIANNE+MURIEL
        for ts in TRAINING_3:
            _,_,f1,_ = f1scoremargin(gtl,videos,TRAINING_3,f1_margin)
            score_folds.append(f1)

        training_mean_score = statistics.mean(score_folds)
        scores.append({"model":model_id, "param":p_idx, "score":training_mean_score})

type_col=["model","param","score"]
resexcel=pd.DataFrame()
resexcel["type"]=type_col

best_models=[]
best_score=0
for i in scores:
    if i["score"] > best_score:
        best_models=[]
        best_models.append(i)
        best_score=i["score"]
    elif i["score"] == best_score:
        best_models.append(i)


best_scores=[]
for best_idx,best in enumerate(best_models):

    best_score=[]

    _,_,f1,_ = f1scoremargin(gtl,models[best["model"]][best["param"]], CORA, f1_margin)
    best_score.append(f1)

    _,_,f1,_ = f1scoremargin(gtl,models[best["model"]][best["param"]], MARIANNE, f1_margin)
    best_score.append(f1)

    _,_,f1,_ = f1scoremargin(gtl,models[best["model"]][best["param"]], MURIEL, f1_margin)
    best_score.append(f1)


    best_scores.append({"model":model_id, "param":p_idx, "score":statistics.mean(best_score)})
    resexcel[best_idx]=[model_id,p_idx,statistics.mean(best_score)]
#gg
resexcel.to_excel("outputFile/LODO.xlsx")
print("lodo")
for i in best_scores:
    print(i)


lodo
{'model': 19, 'param': 242, 'score': 0.7129275024011865}
{'model': 19, 'param': 242, 'score': 0.7129275024011865}


In [10]:
# USE ALL TO BRUTE FORCE AND SCORING
f1_margin=100
scores=[]
for model_id,params in enumerate(models): #1 model = 243 params <- 20 model
    for p_idx,videos in enumerate(params): # 21 video = 1 param <- 243 params
        score_one = []
        best_one=0
        vid_b_one=0
        worst_one=1
        vid_w_one=0
        neg_one=0
        for video_idx,video in enumerate(videos): # 1 prediction = 1 video <- 1 param = 21 video

      
            gt = gtl[video_idx]
            _,_,f1,_,_,_=f1scoremargin(gt,video,f1_margin)
            score_one.append(f1)
            if f1 < 0.5:
                neg_one+=1
            if f1 < worst_one:
                worst_one=f1
                vid_w_one = video_idx
            if f1 > best_one:
                best_one=f1
                vid_b_one = video_idx
            

            

        scores.append({"model":model_id, "param":p_idx, "score":statistics.mean(score_one), "neg":neg_one, "best":best_one,"video_best":vid_b_one, "worst":worst_one, "video_worst":vid_w_one})

type_col=["model","param","score"]
resexcel=pd.DataFrame()
resexcel["type"]=type_col
best_models=[]
best_score=0
for i in scores:

    # if better, reset list and append
    if i["score"] > best_score:
        best_models=[]
        best_models.append(i)
        best_score = i["score"]
    # if equal, append to the list
    elif i["score"] == best_score:
        best_models.append(i)

    #resexcel[best_idx]=[model_id,p_idx,statistics.mean(best_score)]

resexcel.to_excel("outputFile/SCOREALL.xlsx")
print("lovo")
for i in best_models:
    print(i)



TypeError: f1scoremargin() missing 1 required positional argument: 'margin'